In [135]:
# Terraform an RDS instance: rds.tf
# Output the RDS connection props to a config file
# SQL script to make tables in RDS instance
# Python script to pull data, write to file, and copy to RDS instance

In [171]:
import requests
import json

In [212]:
API_KEY = 'DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b'
BASE_URL = 'https://api.open.fec.gov/v1'
DEFAULT_PARAMS = {
  'per_page': 100
}
ENDPOINTS = {
  'candidates': 'candidates',
  'committees': 'committees',
  'donations': 'schedules/schedule_a'
}

# Keys to pull from the FEC response objects
CANDIDATE_ATTRIBUTES = ['candidate_id', 'name', 'cycles', 'district_number', 'election_districts', 'incumbent_challenge', 'party_full', 'state', 'office_full']
COMMITTEE_ATTRIBUTES = ['committee_id', 'committee_type_full', 'designation_full', 'name', 'organization_type_full', 'state', 'party_full''committee_type_full']
DONATION_ATTRIBUTES = ['contribution_receipt_date', 'contribution_receipt_amount', 'contributor_state', 'contributor_id', 'contributor_name', 'pdf_url', 'sub_id']
session = requests.Session()

In [187]:
def format_params(param, value):
  return_params = []
  if isinstance(value, list):
    for value_option in value:
      return_params.append('{}={}'.format(param, value_option))
  else:
    return_params.append('{}={}'.format(param, value))
  return return_params

def get_url_params(params):
  default_params = DEFAULT_PARAMS.copy()
  default_params.update(params)
  url_params = []
  for param, value in default_params.items():
    url_params.extend(format_params(param, value))
  return url_params

In [207]:
def get_url(base_url, endpoint, api_key, **kwargs):
  # Pass kwargs to add params to the URL for filters, etc.
  # See params for your endpoint in the docs: https://api.open.fec.gov/developers/
  url_params = get_url_params(kwargs.copy())
  endpoint = '/'.join([base_url, endpoint])
  query_url = endpoint + '?api_key={}&'.format(api_key) + '&'.join(url_params)
  print(query_url)
  return query_url

In [140]:
def get_data(endpoint, **kwargs):
  endpoint = ENDPOINTS[endpoint]
  url = get_url(BASE_URL, endpoint, API_KEY, **kwargs)
  response = session.get(url)
  return response.json()

In [214]:
def format_record(record, keys):
  return {key: value for key, value in record.items() if key in keys}

In [199]:
def get_object_ids(objects, id_key):
  return list(map(lambda object: object[id_key], objects))

In [193]:
# Example usage:

# Start with candidates
candidate_data = get_data('candidates', party='DEM', name='CLINTON, HILLARY RODHAM')['results']
candidates = [format_record(candidate, CANDIDATE_ATTRIBUTES) for candidate in candidate_data]
candidates

[{'candidate_id': 'S0NY00188',
  'cycles': [2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018],
  'district_number': 0,
  'election_districts': ['00', '00', '00'],
  'incumbent_challenge': 'C',
  'name': 'CLINTON, HILLARY RODHAM',
  'office_full': 'Senate',
  'party_full': 'DEMOCRATIC PARTY',
  'state': 'NY'},
 {'candidate_id': 'P00003392',
  'cycles': [2006, 2008, 2010, 2012, 2014, 2016, 2018],
  'district_number': 0,
  'election_districts': ['00', '00'],
  'incumbent_challenge': 'O',
  'name': 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE',
  'office_full': 'President',
  'party_full': 'DEMOCRATIC PARTY',
  'state': 'US'}]

In [200]:
# Loop through candidates for Committee data
committees_data = get_data('committees', candidate_id=get_object_ids(candidates, 'candidate_id'))['results']
committees = [format_record(committee, COMMITTEE_ATTRIBUTES) for committee in committees_data]
committees

[{'committee_id': 'C00388363',
  'committee_type_full': 'Party - Nonqualified',
  'designation_full': 'Joint fundraising committee',
  'name': 'CLINTON-DSCC VICTORY FUND',
  'organization_type_full': None,
  'state': 'DC'},
 {'committee_id': 'C00348771',
  'committee_type_full': 'Single Candidate Independent Expenditure',
  'designation_full': 'Unauthorized',
  'name': 'COALITION FOR A BETTER AMERICA',
  'organization_type_full': None,
  'state': 'MN'},
 {'committee_id': 'C00390468',
  'committee_type_full': 'Single Candidate Independent Expenditure',
  'designation_full': 'Unauthorized',
  'name': 'DRAFT HILLARY FOR PRESIDENT 2004',
  'organization_type_full': None,
  'state': 'VA'},
 {'committee_id': 'C00358895',
  'committee_type_full': 'Senate',
  'designation_full': 'Principal campaign committee',
  'name': 'FRIENDS OF HILLARY',
  'organization_type_full': None,
  'state': 'DC'},
 {'committee_id': 'C00570978',
  'committee_type_full': 'PAC - Nonqualified',
  'designation_full': 'U

In [217]:
# Loop through committees for donation data
donation_data = []
for committee_id in get_object_ids(committees, 'committee_id'):
  donations_subset = [format_record(donation, DONATION_ATTRIBUTES) for donation in get_data('donations', committee_id=committee_id, is_individual="false")['results']]
  donation_data.extend(donations_subset)

https://api.open.fec.gov/v1/schedules/schedule_a?api_key=DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b&per_page=100&committee_id=C00388363&is_individual=false
https://api.open.fec.gov/v1/schedules/schedule_a?api_key=DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b&per_page=100&committee_id=C00348771&is_individual=false
https://api.open.fec.gov/v1/schedules/schedule_a?api_key=DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b&per_page=100&committee_id=C00390468&is_individual=false
https://api.open.fec.gov/v1/schedules/schedule_a?api_key=DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b&per_page=100&committee_id=C00358895&is_individual=false
https://api.open.fec.gov/v1/schedules/schedule_a?api_key=DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b&per_page=100&committee_id=C00570978&is_individual=false
https://api.open.fec.gov/v1/schedules/schedule_a?api_key=DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b&per_page=100&committee_id=C00431569&is_individual=false
https://api.open.fec.gov/v1/schedules/schedule_a?api_key=DotDawmB1KyneOh1BjX

In [218]:
donation_data[:10]

[{'contribution_receipt_amount': 183.46,
  'contribution_receipt_date': '2017-06-19T00:00:00',
  'contributor_id': None,
  'contributor_name': 'AT&T',
  'contributor_state': 'WI',
  'pdf_url': 'http://docquery.fec.gov/cgi-bin/fecimg/?201707200200229519',
  'sub_id': '2072420171429954288'},
 {'contribution_receipt_amount': 6.36,
  'contribution_receipt_date': '2017-09-29T00:00:00',
  'contributor_id': None,
  'contributor_name': 'AT&T',
  'contributor_state': 'WI',
  'pdf_url': 'http://docquery.fec.gov/cgi-bin/fecimg/?201710180200325148',
  'sub_id': '2102220171458891974'},
 {'contribution_receipt_amount': 8.13,
  'contribution_receipt_date': '2017-10-31T00:00:00',
  'contributor_id': None,
  'contributor_name': 'AT&T',
  'contributor_state': 'WI',
  'pdf_url': 'http://docquery.fec.gov/cgi-bin/fecimg/?201802080200157362',
  'sub_id': '2021220181504498968'},
 {'contribution_receipt_amount': 93.37,
  'contribution_receipt_date': '2017-11-01T00:00:00',
  'contributor_id': None,
  'contribu

In [ ]:
# Format the data
def format_record(record):
  pass

# Write data to file
def write_to_file(record):
  pass

# Python script to read in the rds config file and run copy command 
def copy_to_rds(file):
  pass

In [106]:
### IGNORE EVERYTHING BELOW THIS FOR NOW
### JUST GET RAW DATA INTO THE DB. 
### WE CAN NORMALIZE AND UPDATE THE SCHEMA LATER

def split_running_mates(office, campaign_name):
  candidates = campaign_name.split('/')
  candidate = candidates[0].strip(' ')
  running_mate = candidates[1].strip(' ') if len(candidates) == 2 else 'N/A'
  return (candidate, running_mate)

In [154]:
def format_candidate(candidate_id, candidate):
    if ',' in candidate:
      names = candidate.split(',')
      print('names: ', names)
      last_name = names[0]
      other_names = names[1].strip(' ').split(' ')
      print('other names:', other_names)      
      first_name = other_names[0]
      if len(other_names) > 1:
        middle_names = ' '.join(other_names[1:])
      else:
        middle_names = ''
    else:
      names = candidate.split(' ')
      first_name = names[0]
      last_name = names[-1]
      middle_names = ' '.join(names[1:-1])
    return {'id': candidate_id, 'first_name': first_name, 'middle_names': middle_names, 'last_name': last_name}
      

[{'campaign_name': 'CLINTON, HILLARY RODHAM',
  'candidate': 'CLINTON, HILLARY RODHAM',
  'id': 'S0NY00188-2000',
  'office': 'Senate',
  'running_mate': 'N/A'},
 {'campaign_name': 'CLINTON, HILLARY RODHAM',
  'candidate': 'CLINTON, HILLARY RODHAM',
  'id': 'S0NY00188-2006',
  'office': 'Senate',
  'running_mate': 'N/A'},
 {'campaign_name': 'CLINTON, HILLARY RODHAM',
  'candidate': 'CLINTON, HILLARY RODHAM',
  'id': 'S0NY00188-2012',
  'office': 'Senate',
  'running_mate': 'N/A'},
 {'campaign_name': 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE',
  'candidate': 'CLINTON, HILLARY RODHAM',
  'id': 'P00003392-2008',
  'office': 'President',
  'running_mate': 'TIMOTHY MICHAEL KAINE'},
 {'campaign_name': 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE',
  'candidate': 'CLINTON, HILLARY RODHAM',
  'id': 'P00003392-2016',
  'office': 'President',
  'running_mate': 'TIMOTHY MICHAEL KAINE'}]

In [ ]:
def format_campaign_data(candidate_data):
  campaigns = []
  for record in candidate_data:
    election_years = record.get('election_years', [])
    office = record.get('office_full', 'N/A')
    campaign_name = record.get('name', 'Unknown')
    candidate_id = record.get('candidate_id', '')
    for election_year in election_years:
      campaign = {
        'id': candidate_id + '-' + str(election_year), 
        'campaign_name': campaign_name,
        'office': office
      }
      candidate, running_mate = split_running_mates(office, campaign_name)
      campaign['candidate'] = candidate
      campaign['running_mate'] = running_mate
      campaigns.append(campaign)
  return campaigns  

In [ ]:

campaigns = format_campaign_data(candidates)
unique_candidates = set([campaign['candidate'] for campaign in campaigns] + [campaign['running_mate'] for campaign in campaigns if campaign['running_mate'] != 'N/A'])
labeled_candidates = [format_candidate(index, candidate) for index, candidate in enumerate(unique_candidates)]
labeled_candidates